In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
from Model import *
from sklearn.metrics import classification_report
import tensorflow_addons as tfa
from keras.utils import io_utils
from pandarallel import pandarallel
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
pandarallel.initialize(progress_bar=True, nb_workers=16)
tqdm.pandas()

2023-02-25 19:50:27.407938: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-25 19:50:27.496188: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-25 19:50:27.984224: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/dev/miniconda3/envs/goodbook/lib/
2023-02-25 19:50:27.988950: W tensorflow/compi

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
train = pd.read_csv("../../dataset/goodreads_train.csv")
test = pd.read_csv("../../dataset/goodreads_test.csv")
vocabulary = np.load('../../vocabulaires/voc_without_std_word_count_5.npy', allow_pickle=True)
#train['review_text'] = train['review_text'].str.replace('[^\w\s]','')
train = shuffle(train)

In [3]:
def convert_rating(x):
    if x > 3:
        return 1
    else:
        return 0
train["rating"] = train["rating"].apply(convert_rating)

In [4]:
inputs_data = train[['review_text']]
#outputs_data = keras.utils.to_categorical(train['rating'], num_classes=2)
train_in, validation_in, train_out, validation_out = train_test_split(inputs_data, train['rating'], test_size=0.2)

train_in = [train_in['review_text']]
validation_in = [validation_in['review_text']]

In [5]:
inputs1 = keras.Input(shape=(1,), dtype=tf.string)  # text
vectorize_layer = keras.layers.TextVectorization(
        standardize='lower_and_strip_punctuation',
        split='whitespace',
        output_mode='int',
        output_sequence_length=352,
        vocabulary=vocabulary
    )(inputs1)
vectorize_layer = keras.layers.Flatten()(vectorize_layer)
output = keras.layers.Dense(1, activation='sigmoid')(vectorize_layer)

model_lineair = keras.Model(inputs=[inputs1], outputs=output, name="ml")

2023-02-25 19:50:45.063892: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-25 19:50:45.097947: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-25 19:50:45.097990: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-25 19:50:45.098772: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them i

In [6]:
model_lineair.compile(optimizer=keras.optimizers.Adamax(learning_rate=0.01),
                           loss=keras.losses.mse,
                           metrics=[keras.metrics.categorical_accuracy]
                        )

In [7]:
def scheduler(epoch, lr):
    if epoch < 1:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
chekpoint = keras.callbacks.ModelCheckpoint(f'checkpoint/model_lineair.h5',
            monitor='val_f1_score', mode='max', save_best_only=True)

sheduler = keras.callbacks.LearningRateScheduler(scheduler,0)
stop_nan = keras.callbacks.TerminateOnNaN()

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"logs/model_lineair/")
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"logs/{model.name}/{seed}/{epsilone}/")
model_lineair.fit(x=train_in, y=train_out, validation_data=(validation_in, validation_out), batch_size=64, epochs=15, callbacks=[stop_nan,tensorboard_callback, chekpoint, sheduler])

Epoch 1/15


2023-02-25 19:50:47.554145: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f68739c37b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-25 19:50:47.554179: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2023-02-25 19:50:47.558002: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-02-25 19:50:47.570636: W tensorflow/compiler/xla/service/gpu/nvptx_helper.cc:56] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  /usr/local/cuda-11.2
  /usr/local/cuda
  .
You can choose the search directory by setting xla_gpu_cuda_data_dir in HloModule's DebugO

InternalError: Graph execution error:

Detected at node 'StatefulPartitionedCall_1' defined at (most recent call last):
    File "/home/dev/miniconda3/envs/goodbook/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/dev/miniconda3/envs/goodbook/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/dev/miniconda3/envs/goodbook/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/dev/miniconda3/envs/goodbook/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/dev/miniconda3/envs/goodbook/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home/dev/miniconda3/envs/goodbook/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_f

In [ ]:
model = unet5.Model(vocabulary)
model.model.load_weights('checkpoint/unet5/')

In [ ]:
res = model.model.predict(train_in)

In [ ]:
val_res = model.model.predict(validation_in)

In [ ]:
val_out = np.argmax(validation_out, axis=1)
train_out = np.argmax(train_out, axis=1)


In [ ]:
test[['read_at','date_added','date_updated' ,'started_at']] = test[['read_at','date_added','date_updated' ,'started_at']].parallel_applymap(convert_timestamp)

test_data = [test['review_text'], test['n_comments'], test['n_votes'], test['read_at'], test['date_added'], test['date_updated'], test['started_at']]
restest = model.model.predict(test_data)

In [ ]:
ff = []
for line in tqdm(val_res):
    tmp = -2
    category = None
    for i in (range(6)):
        if line[i] > tmp:
            category = i
            tmp = line[i]
    ff.append(category)
val_data = np.array(ff)

ff = []
for line in tqdm(restest):
    tmp = -2
    category = None
    for i in (range(6)):
        if line[i] > tmp:
            category = i
            tmp = line[i]
    ff.append(category)
test_data = np.array(ff)

ff = []
for line in tqdm(res):
    tmp = -2
    category = None
    for i in (range(6)):
        if line[i] > tmp:
            category = i
            tmp = line[i]
    ff.append(category)
train_data = np.array(ff)

In [ ]:
print(classification_report(train_out, train_data))

In [ ]:
print(classification_report(val_out, val_data))

In [ ]:
test['rating'] = test_data

id = test['review_id'].to_numpy()
rating = test['rating'].to_numpy()
df = pd.DataFrame( columns=['review_id', 'rating'])
df['review_id'] = id
df['rating'] = rating

In [ ]:
df.to_csv('submission_unet5_embedding_class_weights_model.csv',index=False)

In [ ]:
model.model.save('unet5')